In [20]:
import os
from typing import Dict, List
from groq import Groq

# Get a free API key from https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = "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"

LLAMA3_70B_INSTRUCT = "llama3-70b-8192"
LLAMA3_8B_INSTRUCT = "llama3-8b-8192"

DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

client = Groq()


In [21]:
from typing import List, Optional
import json

from pydantic import BaseModel
from groq import Groq

groq = Groq()

class Spells(BaseModel):
    club_name: str
    manager: str
    end_of_spell: str
    start_of_spell: str
    league: str
    tier: str
    sacked: Optional[bool]
    probability_of_sacking_true: Optional[float]


def get_spell(club_name: str,end_of_spell: str) -> Spells:
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a football statistician that outputs football facts about managerial spells in JSON.\n"
                # Pass the json schema to the model. Pretty printing improves results.
                f" The JSON object must use the schema: {json.dumps(Spells.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"Fetch football facts about the managerial spells at {club_name} which ended in {end_of_spell}.",
            },
        ],
        model=DEFAULT_MODEL,
        temperature=0,
        top_p= 0.9,
        # Streaming is not supported in JSON mode
        stream=False,
        # Enable JSON mode by setting the response format
        response_format={"type": "json_object"},
    )
    return Spells.model_validate_json(chat_completion.choices[0].message.content)


spell = get_spell("Manchester United", "1965")
#print(spell.club_name)


# Extracting information from managerial spells

In [27]:
progress_bar.close()

  0%|          | 32/70600 [01:49<66:46:17,  3.41s/it]


In [28]:
import pandas as pd
from tqdm import tqdm
from groq import BadRequestError
inputs = pd.read_csv("LLM_input.csv")

# Create a progress bar
progress_bar = tqdm(total=len(inputs))

# Iterate over the rows of the csv file
for i, row in inputs.iterrows():
    club = row["club_name"]
    date = row["end_of_spell"]
    try: 
        spell = get_spell(club, date)
        print(spell.club_name)
        print(spell.manager)
        print(spell.end_of_spell)
        print(spell.start_of_spell)
        print(spell.league)
        print(spell.tier)
        print(spell.sacked)
        print(spell.probability_of_sacking_true)
        print("\n")
        # Save the output to a new dataframe
        inputs.loc[i, "manager"] = spell.manager
        inputs.loc[i, "league"] = spell.league
        inputs.loc[i, "tier"] = spell.tier
        inputs.loc[i, "sacked"] = spell.sacked
        inputs.loc[i, "probability_of_sacking_true"] = spell.probability_of_sacking_true
    except BadRequestError:
        inputs.loc[i, "manager"] = "Bad Response"    
    
    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()
    

  0%|          | 1/70600 [00:01<25:11:03,  1.28s/it]

West Ham United
David Moyes
2018-05-16
2017-11-07
Premier League
1
False
None




  0%|          | 2/70600 [00:02<24:05:26,  1.23s/it]

Sunderland
David Moyes
2017-05-22
2016-07-23
Premier League
1
True
0.85




  0%|          | 3/70600 [00:03<18:47:21,  1.04it/s]

...

  0%|          | 47/70600 [01:29<60:12:49,  3.07s/it]

Limerick FC
Eoin Hand
1992-06-30
1991-07-01
League of Ireland Premier Division
1
True
0.8




  0%|          | 48/70600 [01:32<60:13:37,  3.07s/it]

West Bromwich Albion
Brian Talbot
1991-06-30
1988-06-01
English Football League
Second Division
True
0.85




  0%|          | 49/70600 [01:34<54:08:28,  2.76s/it]

In [24]:
inputs.head()

,club_name,end_of_spell,manager,league,tier,sacked,probability_of_sacking_true
0,West Ham,2018-05-16,David Moyes,Premier League,1,False,None
1,Sunderland,2017-05-22,David Moyes,Premier League,1,True,0.85
2,Real Sociedad,2015-11-09,David Moyes,La Liga,1,True,0.85
3,Man Utd,2014-04-22,David Moyes,Premier League,1,True,0.85
4,Everton,2013-06-30,David Moyes,Premier League,1,False,None
